## Image Caption Generator


*YAPILACAKLAR*

- load_dataset için Huggingface authentication gibi bişiler istiyor, ya onu yaparız ya da farklı bi library/method kullanırız.

- Bir an önce bug ları çözmek lazım load_dataset e kadar sorun yok an itibariyle inş, kontrol edebilirseniz süper olur.

Yapcazzz bu işiii <3


![image.png](attachment:image.png)


Use of Google Colab is advised

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install system dependencies
!sudo apt-get update
!sudo apt-get install -y build-essential
!sudo apt-get install -y python3-dev

# Create requirements.txt file
with open('requirements.txt', 'w') as f:
    f.write('''
torch==2.0.0
torchvision==0.15.1
torchaudio
torchtext==0.15.1
transformers
datasets
Pillow
numpy
opencv-python-headless
tqdm
requests
fsspec==2024.9.0
''')
    #git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI ultralytics==8.0.3

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [12]:
#rm -rf /content/*

In [3]:
#!git clone https://github.com/ultralytics/yolov5

#Uncomment below if you are using Google Colab.
!pip install -r requirements.txt

#!pip install "git+https://github.com/salaniz/pycocoevalcap.git"
!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"
#!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
#!pip install ultralytics==8.0.3  # Specify a stable YOLOv5 version
#!pip install git+https://github.com/ultralytics/yolov5


  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-install-xn16qmci/pycocotools_5c029779dc424f4da3bb9f4aa14bffa2
  Running command git clone --filter=blob:none --quiet https://github.com/philferriere/cocoapi.git /tmp/pip-install-xn16qmci/pycocotools_5c029779dc424f4da3bb9f4aa14bffa2
  Resolved https://github.com/philferriere/cocoapi.git to commit 2929bd2ef6b451054755dfd7ceb09278f935f7ad
  Preparing metadata (setup.py) ... done


In [23]:
#!git clone https://github.com/ultralytics/yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.


# Inital Setup

In [4]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import os
from PIL import Image
import numpy as np
import requests
from io import BytesIO
from datasets import load_dataset
from tqdm import tqdm
import cv2
import pycocoevalcap
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
from collections import Counter
# for metrics, COCO API metrics is used
import json
from pycocoevalcap import bleu, meteor, rouge, spice
#from pycocoevalcap.evalcap import COCOEvalCap

# Load YOLOv5 for feature extraction
import torch.hub

In [5]:
# setting the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Pre-Trained Encoder (Yolo-v5) 😎

In [4]:
""" -NOT NEEDED ANYMORE- This is for local storage of the YOLO weights, we are uploading the model from github with the next code block.
%cd yolov5
!wget https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt # get the pre-trained weights
"""

/content/yolov5


In [6]:
# Load pre-trained YOLOv5 model from Torch Hub
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).eval().to(device)

#This function pulls models from GitHub repositories and is commonly used for loading pre-trained models. When you specify 'ultralytics/yolov5', it attempts to fetch the YOLOv5 repository from GitHub.


#model = YOLO('yolov8s.pt').to('cuda') if torch.cuda.is_available() else YOLO('yolov8s.pt')
#yolov5s.pt, yolov5m.pt, yolov5l.pt, yolov5x.pt are pretrained YOLOv5 models with different sizes (s for small, m for medium, l for large, x for extra-large).



Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-13 Python-3.10.12 torch-2.0.0+cu117 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [9]:
# Download Dataset
%mkdir -p coco
%cd coco  # load evertything inside 'coco' folder
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!wget http://images.cocodataset.org/zips/train2017.zip
!unzip train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip

[Errno 2] No such file or directory: 'coco # load evertything inside coco folder'
/content
--2024-11-13 20:11:42--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.27.137, 3.5.29.195, 52.217.205.73, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.27.137|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  17.1MB/s    in 17s     

2024-11-13 20:11:59 (14.6 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflatin

In [16]:
model

In [17]:
# Define transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
# Load COCO Dataset (captions and images)

# The COCO dataset is already pre-split into train, validation, and test subsets by its creators, and Hugging Face provides access to these splits directly.

 # Load the training split of the COCO dataset
train_dataset = load_dataset("coco", split='train')

# Load the validation split
val_dataset = load_dataset("coco", split='validation')

# Load the test split
test_dataset = load_dataset("coco", split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetNotFoundError: Dataset 'coco' doesn't exist on the Hub or cannot be accessed.

In [ ]:
class CocoDataset(Dataset): # <start> cat sat on the mat <end> -> {0 32 24 34 3 3 1 }
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.annotations = load_dataset('json', data_files=annotation_file)['train']
        self.tokenizer = get_tokenizer("basic_english")  # Tokenizer from torchtext

        # Build vocabulary
        self.vocab = self.build_vocab()

    def build_vocab(self):
        counter = Counter()
        for annotation in tqdm(self.annotations):
            caption = annotation['caption']
            tokens = self.tokenizer(caption.lower())
            counter.update(tokens)
        vocab = build_vocab_from_iterator([counter], specials=["<pad>", "<unk>", "<bos>", "<eos>"])
        vocab.set_default_index(vocab["<unk>"])  # Out-of-vocabulary words will be mapped to <unk>
        return vocab

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations[idx]['file_name'])
        image = Image.open(img_path).convert('RGB')
        caption = self.annotations[idx]['caption']
        tokens = self.tokenizer(caption.lower())

        # Convert caption tokens to indices
        caption_indices = [self.vocab['<bos>']] + [self.vocab[token] for token in tokens] + [self.vocab['<eos>']]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(caption_indices)


In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        self.cnn = yolov5.models.common.AutoShape(yolov5.load("yolov5s"))
        self.fc = nn.Linear(1000, embed_size)

    def forward(self, images):
        features = self.cnn(images)  # Get features from YOLOv5
        features = features.view(features.size(0), -1)
        features = self.fc(features)
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers,batch_first=True)#, bi-directional=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, features, captions):
        embeddings = self.embedding(captions)
        inputs = torch.cat((features.unsqueeze(1), embeddings), dim=1)
        lstm_out, _ = self.lstm(inputs)
        output = self.fc(lstm_out)
        return output

class ImageCaptioningModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(ImageCaptioningModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs


In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    for images, captions in dataloader:
        images = torch.stack(images).to(device)
        captions = torch.stack(captions).to(device)

        optimizer.zero_grad()
        outputs = model(images, captions[:, :-1])  # Exclude last token in captions
        loss = criterion(outputs.view(-1, outputs.size(2)), captions[:, 1:].contiguous().view(-1))  # Teacher forcing
        loss.backward()
        optimizer.step()

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, captions in dataloader:
            images = torch.stack(images).to(device)
            captions = torch.stack(captions).to(device)

            outputs = model(images, captions[:, :-1])  # Exclude last token in captions
            loss = criterion(outputs.view(-1, outputs.size(2)), captions[:, 1:].contiguous().view(-1))  # Teacher forcing
            total_loss += loss.item()

    return total_loss / len(dataloader)


In [ ]:
def generate_caption(image, model, vocab, device):
    model.eval()
    image = transform(image).unsqueeze(0).to(device)
    features = model.encoder(image)

    # Generate caption
    caption = ['<bos>']
    for _ in range(50):  # Maximum caption length
        input_caption = torch.tensor([vocab[token] for token in caption]).unsqueeze(0).to(device)
        outputs = model.decoder(features, input_caption)
        _, predicted = outputs.max(2)
        predicted_word = vocab.lookup_token(predicted[0, -1].item())
        caption.append(predicted_word)
        if predicted_word == '<eos>':
            break

    return ' '.join(caption[1:-1])  # Removing <bos> and <eos>

# Load trained model and generate a caption
model.load_state_dict(torch.load('image_captioning_model.pth'))
image = Image.open("path_to_image.jpg")  # Replace with your image path
caption = generate_caption(image, model, train_dataset.vocab, device)
print(f"Generated Caption: {caption}")


In [ ]:
# Hyperparameters
embed_size = 256
hidden_size = 512
num_epochs = 10
learning_rate = 0.001

# Instantiate Encoder, Decoder, and Model
encoder = EncoderCNN(embed_size).to(device)
decoder = DecoderRNN(embed_size, hidden_size, len(train_dataset.vocab)).to(device)
model = ImageCaptioningModel(encoder, decoder).to(3)

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Training Loop
for epoch in range(num_epochs):
    train(model, train_loader, optimizer, criterion, device)
    val_loss = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}")

# Save the trained model
torch.save(model.state_dict(), 'image_captioning_model.pth')

In [ ]:
caption = generate_caption(encoder, decoder, 'path/to/your/image.jpg', vocab, transform, device)
print("Generated Caption:", caption)

In [ ]:


def evaluate_captioning_model(generated_captions, coco_annotation_file='annotations/captions_val2017.json'):
    """
    Evaluate the image captioning model using COCO evaluation metrics: BLEU, METEOR, ROUGE, CIDEr.

    Parameters:
        generated_captions (dict): Dictionary of generated captions with image_ids as keys.
        coco_annotation_file (str): Path to COCO annotations file.

    Returns:
        dict: Dictionary containing BLEU, METEOR, ROUGE, CIDEr scores.
    """
    # Load the COCO dataset annotations (reference captions)
    coco = COCO(coco_annotation_file)

    # Create a dictionary for the generated captions (image_id -> caption)
    coco_results = []
    for image_id, caption in generated_captions.items():
        coco_results.append({
            'image_id': image_id,
            'caption': caption
        })

    # Save the generated captions in a temporary file
    with open('generated_captions.json', 'w') as f:
        json.dump(coco_results, f)

    # Load the results into COCO's evaluation API
    coco_results = coco.loadRes('generated_captions.json')

    # Set up the evaluation
    coco_eval = COCOEvalCap(coco, coco_results)
    coco_eval.evaluate()

    # Extract and return the metrics (BLEU, METEOR, ROUGE, CIDEr)
    metrics = coco_eval.eval
    return metrics

# Example: Generated captions for some images
generated_captions = {
    12345: "A man in a black shirt is riding a bike.",
    67890: "A dog running through the grass.",
    11223: "A woman holding a book in her hand."
}

# Evaluate the generated captions
metrics = evaluate_captioning_model(generated_captions)
print("Evaluation Metrics:", metrics)